In [1]:
import torch

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [4]:
device = torch.device("cpu")

In [5]:
from transformers import TextDataset, DataCollatorForLanguageModeling, GPT2Model, AdamW
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import torch.optim as optim

c:\Users\hasee\anaconda3\envs\new_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import csv

# Assuming your CSV file is named 'your_file.csv'
csv_file_path = 'Transformed_Chatbot_Dataset_Shakespearean.csv'

# List to store question-answer pairs
qa_pairs = []

# Read CSV file and extract question-answer pairs
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        # Assuming your CSV columns are named "Questions" and "Answers"
        question = row['Prompt']
        answer = row['Shakespearean Reply']
        qa_pair = [f"Q: {question} A: {answer}"]
        qa_pairs.append(qa_pair)

In [7]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

model.config.pad_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

text = ""
for question in qa_pairs:
    text += f"{question} {tokenizer.eos_token}"

In [21]:
#input_ids = tokenizer(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
#input_ids = tokenizer.encode(text,, add_special_tokens=True, return_tensors="pt")
input_ids = tokenizer.encode(text,padding=True, truncation=True, add_special_tokens=True, return_tensors="pt")

In [22]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [37]:
qa_pairs[10]

['Q: good luck with school. A: thank thou very much.']

In [24]:
model.to(device)
model.train()

for epoch in range(51):
    optimizer.zero_grad()
    loss = model(input_ids.to(device), labels=input_ids.to(device))[0]
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(f"Epoch {epoch}, Loss {loss.item()}")

Epoch 0, Loss 0.21510645747184753
Epoch 1, Loss 2.108739137649536
Epoch 2, Loss 1.0774857997894287
Epoch 3, Loss 0.9498881697654724
Epoch 4, Loss 0.7105492949485779
Epoch 5, Loss 0.541527271270752
Epoch 6, Loss 0.4650467038154602
Epoch 7, Loss 0.42432212829589844
Epoch 8, Loss 0.36263027787208557
Epoch 9, Loss 0.3202010691165924
Epoch 10, Loss 0.29578471183776855
Epoch 11, Loss 0.26422759890556335
Epoch 12, Loss 0.23515275120735168
Epoch 13, Loss 0.21736979484558105
Epoch 14, Loss 0.2072908580303192
Epoch 15, Loss 0.1820899397134781
Epoch 16, Loss 0.1717212200164795
Epoch 17, Loss 0.15771843492984772
Epoch 18, Loss 0.16696590185165405
Epoch 19, Loss 0.14041493833065033
Epoch 20, Loss 0.12990158796310425
Epoch 21, Loss 0.12798650562763214
Epoch 22, Loss 0.12152053415775299
Epoch 23, Loss 0.12383782863616943
Epoch 24, Loss 0.11000888794660568
Epoch 25, Loss 0.11251053959131241
Epoch 26, Loss 0.10238364338874817
Epoch 27, Loss 0.0991755872964859
Epoch 28, Loss 0.10326429456472397
Epoch 29

In [25]:
model.eval()
def generate_response(question):
    # Encode the question using the tokenizer
    input_ids = tokenizer.encode(question + "<|question|>", add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device)
    # Generate the answer using the model
    attention_mask = torch.ones_like(input_ids)
    sample_output = model.generate(input_ids, do_sample=True, max_length=100, top_k=20, top_p=1.0, attention_mask=attention_mask)

    # Decode the generated answer using the tokenizer
    answer = tokenizer.decode(sample_output[0], skip_special_tokens=True)

    # Split the generated answer into individual sentences
    sentences = answer.split(". ")

    # Look for the sentence that contains the answer to the question
    for sentence in sentences:
        if question in sentence:
            return sentence

    # If no sentence contains the answer, return the full generated answer
    return answer

In [39]:
question = "what's the weather going to be like?"
response = generate_response(question)
print(f"{question}\n {response}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


what's the weather going to be like?
 what's the weather going to be like?<|question|> forsooth, i'm fine
